# Step 3: TTI by DALL-E 3
재구성된 text를 토대로, 이를 설명하는 고퀄리티의 image 생성
* 사용 모델: DALL-E

In [6]:
from dotenv import load_dotenv

# 토큰 정보로드
load_dotenv()

True

## Client 생성

In [7]:
import openai

print(f"설치된 버전: {openai.__version__}")

설치된 버전: 1.36.0


In [8]:
import os

openai_api_key = os.getenv("OPENAI_API_KEY")

In [9]:
from openai import OpenAI

client = OpenAI(api_key = openai_api_key)

## 1. 이미지 생성(image.generate)
* 이미지 크기: dall-e-3 = 1024x1024, 1792x1024, 1024x1792
* DALL-E 3 사용 시, "HD"로 설정하면 디테일이 향상됨. 정사각형의 이미지가 가장 빠르게 생성됨.
* 사진요청 개수: 한 번에 1개의 이미지를 요청할 수 있음; 대신 병렬 요청을 통해 더 많은 이미지를 요청할 수 있다.
* prompt로 생성되는 이미지의 퀄리티가 많이 달라진다.

In [5]:
# 텍스트 파일의 내용을 읽어 image_prompt로 설정합니다.
with open('simplified_medical_report.txt', 'r', encoding='utf-8') as file:
    image_prompt = file.read()

# DALL-E 3 모델을 사용하여 이미지를 생성합니다.
response = client.images.generate( #image generation
    model="dall-e-3", #달리2 or 3 사용가능
    prompt=image_prompt,
    size="1024x1024",
    quality="standard",
    n=1,
)

# 생성된 이미지의 URL을 저장합니다.
image_url = response.data[0].url

BadRequestError: Error code: 400 - {'error': {'code': 'content_policy_violation', 'message': 'Your request was rejected as a result of our safety system. Image descriptions generated from your prompt may contain text that is not allowed by our safety system. If you believe this was done in error, your request may succeed if retried, or by adjusting your prompt.', 'param': None, 'type': 'invalid_request_error'}}

>> 이 오류는 DALL-E의 콘텐츠 정책 위반으로 인해 발생한 것으로 보입니다. 오류 메시지에 따르면, 요청한 이미지 설명에 DALL-E의 안전 시스템에서 허용하지 않는 텍스트가 포함되어 있을 가능성이 있습니다.
이 문제를 해결하기 위해 다음과 같은 방법을 시도해 볼 수 있습니다:
프롬프트 수정:
simplified_medical_report.txt 파일의 내용을 검토하고, 잠재적으로 문제가 될 수 있는 내용을 제거하거나 수정합니다. 특히 민감한 의료 정보, 개인 정보, 폭력적이거나 부적절한 내용 등이 있다면 제거해야 합니다.
프롬프트 길이 확인:
DALL-E에는 프롬프트 길이 제한이 있습니다. 파일의 내용이 너무 길다면, 핵심 내용만 추출하여 사용해 보세요.
안전한 버전으로 재작성:
의료 보고서의 내용을 직접 사용하는 대신, 그 내용을 바탕으로 안전하고 일반적인 설명으로 재작성해 보세요.
에러 처리 추가:
코드에 에러 처리를 추가하여 실패 시 대체 프롬프트를 사용하거나 사용자에게 알림을 줄 수 있습니다.
예를 들어, 다음과 같이 코드를 수정할 수 있습니다:

==> 의료 관련 내용이라, DALL-E 3에서 자체적으로 차단한 것으로 보입니다. 

In [10]:
# 파일에서 프롬프트 읽기
with open('simplified_medical_report.txt', 'r', encoding='utf-8') as file:
    original_prompt = file.read()

# 프롬프트 수정 (예: 길이 제한, 민감한 정보 제거)
max_length = 1000  # DALL-E의 실제 제한에 맞게 조정하세요
image_prompt = original_prompt[:max_length]  # 길이 제한
image_prompt = "Create a general medical illustration based on the following description, without including any specific patient information: " + image_prompt

try:
    # DALL-E 3 모델을 사용하여 이미지를 생성합니다.
    response = client.images.generate(
        model="dall-e-3",
        prompt=image_prompt,
        size="1024x1024",
        quality="standard",
        n=1,
    )

    # 생성된 이미지의 URL을 저장합니다.
    image_url = response.data[0].url
    print(f"Image generated successfully. URL: {image_url}")

except openai.BadRequestError as e:
    print(f"Error generating image: {e}")

Error generating image: Error code: 400 - {'error': {'code': 'content_policy_violation', 'message': 'Your request was rejected as a result of our safety system. Image descriptions generated from your prompt may contain text that is not allowed by our safety system. If you believe this was done in error, your request may succeed if retried, or by adjusting your prompt.', 'param': None, 'type': 'invalid_request_error'}}


다운로드 받을 수 있는 이미지의 URL을 출력합니다.

In [ ]:
# 생성된 이미지의 URL을 출력합니다.
print(image_url)

Jupyter Notebook에서 출력하기 위하여 Image를 사용합니다.

In [ ]:
from IPython.display import Image

# 생성된 이미지를 출력합니다.
Image(url=image_url, width=500)

urllib 라이브러리를 사용해 이미지를 다운로드합니다.

In [ ]:
import urllib

# 생성된 이미지를 URL로부터 다운로드하여 저장합니다.
urllib.request.urlretrieve(image_url, "generated_medical_image.jpg")